In [1]:
import openmm.app as app
import openmm as mm
import openmm.unit as unit
import numpy as np
import jax.numpy as jnp
import dmff
from dmff.api.xmlio import XMLIO
from dmff.api.paramset import ParamSet
from dmff.generators.classical import CoulombGenerator, LennardJonesGenerator
from dmff.api.hamiltonian import Hamiltonian
from dmff.operators import ParmedLennardJonesOperator
import parmed

In [2]:
prm_top = parmed.load_file("Lig.top")
dmfftop = dmff.DMFFTopology(from_top=prm_top.topology)

In [3]:
prmop = ParmedLennardJonesOperator()
dmfftop = prmop(dmfftop, gmx_top = prm_top)

In [4]:
prmop.renderLennardJonesXML("init.xml")
xmlio = XMLIO()
xmlio.loadXML("init.xml")
ffinfo = xmlio.parseXML()

In [5]:
cov_mat = dmfftop.buildCovMat()

In [6]:
paramset = ParamSet()
lj_gen = LennardJonesGenerator(ffinfo, paramset)

In [7]:
pairs = []
for ii in range(dmfftop.getNumAtoms()):
    for jj in range(ii+1, dmfftop.getNumAtoms()):
        pairs.append([ii, jj, cov_mat[ii, jj]])
pairs = jnp.array(pairs, dtype=int)

pos = app.PDBFile("GAS.pdb").getPositions(asNumpy=True).value_in_unit(unit.nanometer)
pos = jnp.array(pos)

box = jnp.array([
    [10.0, 0.0, 0.0],
    [0.0, 10.0, 0.0],
    [0.0, 0.0, 10.0]
])

In [8]:
lj_force = lj_gen.createPotential(
    dmfftop, nonbondedMethod=app.NoCutoff, nonbondedCutoff=1.0, args={})
ener_mol = lj_force(pos, box, pairs, paramset)
print("JAX mol1: ", ener_mol)

JAX mol1:  21.271616
